In [ ]:
-- ideas-jar-db-creation.sql
-- Database schema for Ideas Jar application

-- Drop tables if they exist to avoid conflicts
DROP TABLE IF EXISTS idea_audio;
DROP TABLE IF EXISTS ideas;
DROP TABLE IF EXISTS users;

-- Create users table
CREATE TABLE users (
    id SERIAL PRIMARY KEY,
    username VARCHAR(50) NOT NULL UNIQUE,
    email VARCHAR(100) NOT NULL UNIQUE,
    password_hash VARCHAR(255) NOT NULL,
    created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
);

-- Create ideas table
CREATE TABLE ideas (
    id SERIAL PRIMARY KEY,
    user_id INTEGER NOT NULL REFERENCES users(id) ON DELETE CASCADE,
    content TEXT NOT NULL,
    is_voice BOOLEAN NOT NULL DEFAULT FALSE,
    created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
);

-- Create idea_audio table to store audio recordings
CREATE TABLE idea_audio (
    id SERIAL PRIMARY KEY,
    idea_id INTEGER NOT NULL REFERENCES ideas(id) ON DELETE CASCADE,
    audio_data BYTEA NOT NULL,
    duration_seconds INTEGER,
    transcription_status VARCHAR(20) DEFAULT 'pending',
    created_at TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
);

-- Create indexes for performance
CREATE INDEX idx_ideas_user_id ON ideas(user_id);
CREATE INDEX idx_ideas_created_at ON ideas(created_at);
CREATE INDEX idx_idea_audio_idea_id ON idea_audio(idea_id);
CREATE INDEX idx_ideas_is_voice ON ideas(is_voice);

-- Create a view for easier querying of ideas with their audio status
CREATE VIEW idea_details AS
SELECT
    i.id,
    i.user_id,
    i.content,
    i.is_voice,
    i.created_at,
    i.updated_at,
    a.id AS audio_id,
    a.duration_seconds,
    a.transcription_status
FROM
    ideas i
LEFT JOIN
    idea_audio a ON i.id = a.idea_id;

-- Insert sample user data
INSERT INTO users (username, email, password_hash)
VALUES ('demo_user', 'demo@example.com', '$2a$10$SomeHashedPasswordExample');

-- Insert sample idea data
INSERT INTO ideas (user_id, content, is_voice)
VALUES
    (1, 'Create a mobile app for tracking daily habits', FALSE),
    (1, 'Design a new logo for the startup project', FALSE),
    (1, 'Research machine learning algorithms for recommendation systems', FALSE),
    (1, 'This is a voice recorded idea about creating something amazing', TRUE),
    (1, 'Voice note about improving user experience in the application', TRUE);

-- Create function to update updated_at timestamp
CREATE OR REPLACE FUNCTION update_modified_column()
RETURNS TRIGGER AS $$
BEGIN
    NEW.updated_at = now();
    RETURN NEW;
END;
$$ language 'plpgsql';

-- Create triggers to automatically update the updated_at column
CREATE TRIGGER update_user_modtime
    BEFORE UPDATE ON users
    FOR EACH ROW
    EXECUTE PROCEDURE update_modified_column();

CREATE TRIGGER update_idea_modtime
    BEFORE UPDATE ON ideas
    FOR EACH ROW
    EXECUTE PROCEDURE update_modified_column();